# License
Copyright (c) 2019 Sopra Steria All rights reserved.
This work is licensed under the terms of the MIT license.  
For a copy, see https://opensource.org/licenses/MIT.

In [17]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

import Orange
from Orange.data import Domain, Table,Instance, DiscreteVariable, ContinuousVariable, StringVariable
from Orange.statistics import contingency
import features
import features_frequency
import os
import sys
from IPython.display import HTML, display

tables={}
temp_tables={}
nb_tables=1
curr_table=1
stats=[['dataset','rows','temp_agg']]
for i in os.listdir(os.getcwd()+"/../data"):
    if i.endswith(".tab"): 
        #print(os.getcwd()+"/../data/"+i)
        ds_table=Table(os.getcwd()+"/../data/"+i)
        new_table=features.generateFeatures(ds_table)
        #print(new_table[0])
        new_tmp_table=features_frequency.temp_byte_features(ds_table)
        #print(new_tmp_table[0])
        if new_table:
            tables[i]=new_table
        if new_tmp_table:
            temp_tables[i]=new_tmp_table
        stats.append([i,len(new_table),len(new_tmp_table)])
        #if curr_table>nb_tables:
        #    break
        curr_table+=1
display(HTML(
'<hr/><h1>Imports</h1><br/><table><tr>{}</tr></table>'.format(
    '</tr><tr>'.join(
        '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in stats)
)))

dataset,rows,temp_agg
2016-05-26.tab,4177,280
2016-05-18.tab,25553,199
2016-05-28.tab,2924,2
2016-05-29.tab,2918,4
2016-06-07.tab,2224,14
2016-06-06.tab,27006,74
2016-05-11.tab,557,155
2016-05-21.tab,3,2
2016-06-01.tab,83599,219
2016-05-13.tab,157,63


In [18]:
import numpy as np, bisect
from sklearn.svm import OneClassSVM
from sklearn import decomposition
import rpca
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from IPython.display import HTML, display


dataset_number=1
dataset_results=[["logfile","events","attack_events","detected","others_detections","nb_dim_err"]]
for dataset_name,dataset in tables.items():
    print(dataset_name)
    tagged=0
    expected=0
    others=0
    population=len(dataset)
    for data in dataset:
        if data['TYPE']=="ATTACK":
            tagged+=1
    X = dataset.X
    result_rpca = rpca.rpca(X,verbose=False, max_iter=10000)
    outliers_rpca=result_rpca[1]
    nb_non_zero=[]
    matched=[["TYPE","DATETIME","USER","URL","REFFERER"]]
    for i,value in enumerate(outliers_rpca):
        if np.count_nonzero(outliers_rpca[i]) > 9:
            if dataset[i]['TYPE']=="ATTACK":
                expected+=1
                nb_non_zero.append(np.count_nonzero(outliers_rpca[i]))
                matched.append([dataset[i]['TYPE'],dataset[i]['DATETIME'],dataset[i]['USER'],dataset[i]['URL'],dataset[i]['REFERRER']])
            else:
                others+=1
    ratio=0
    avg_nb_non_zero=0
    if expected>0:
        avg_nb_non_zero=sum(nb_non_zero)/len(nb_non_zero)
        ratio=expected/tagged*100
    elif tagged==0:
        ratio='NaN'
        avg_nb_non_zero='NaN'
    dataset_results.append([dataset_name,population,tagged,expected,others,avg_nb_non_zero])
    display(HTML(
'<table><tr>{}</tr></table>'.format(
    '</tr><tr>'.join(
        '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in matched)
)))
    dataset_number+=1

display(HTML(
'<hr/><h1>SUMMARY</h1><br/><table><tr>{}</tr></table>'.format(
    '</tr><tr>'.join(
        '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in dataset_results)
)))

2016-05-26.tab


TYPE,DATETIME,USER,URL,REFFERER


2016-05-18.tab


TYPE,DATETIME,USER,URL,REFFERER


2016-05-28.tab


...

2016-05-17.tab


TYPE,DATETIME,USER,URL,REFFERER


2016-05-19.tab


TYPE,DATETIME,USER,URL,REFFERER


logfile,events,attack_events,detected,others_detections,nb_dim_err
2016-05-26.tab,4177,613,0,11,0
2016-05-18.tab,25553,0,0,1,NaN
2016-05-28.tab,2924,2920,0,0,0
2016-05-29.tab,2918,2909,0,0,0
2016-06-07.tab,2224,0,0,0,NaN
2016-06-06.tab,27006,53,0,0,0
2016-05-11.tab,557,0,0,0,NaN
2016-05-21.tab,3,0,0,0,NaN
2016-06-01.tab,83599,4,0,9,0
2016-05-13.tab,157,0,0,1,NaN


In [19]:
import numpy as np
import rpca
for temp_dataset_name,temp_dataset in temp_tables.items():
    #for data in dataset:
    #    if data['TYPE']=="ATTACK":
    #        tagged+=1
    temp_X = temp_dataset.X
    temp_result_rpca = rpca.rpca(temp_X,verbose=False, max_iter=10000)
    temp_outliers_rpca=temp_result_rpca[1]
    print(temp_dataset_name,len(temp_dataset),len(temp_result_rpca[1]))
    nb_non_zero=[]
    matched=[["TYPE","DATETIME","USER","URL","REFFERER"]]
    for j,temp_value in enumerate(temp_outliers_rpca):
        if np.count_nonzero(temp_outliers_rpca[j]) > 9:
            #print(j)
            #print(temp_outliers_rpca[j])

            if dataset[i]['TYPE']=="ATTACK":
                expected+=1
                nb_non_zero.append(np.count_nonzero(outliers_rpca[i]))
                matched.append([dataset[i]['TYPE'],dataset[i]['DATETIME'],dataset[i]['USER'],dataset[i]['URL'],dataset[i]['REFERRER']])
            else:
                others+=1
    ratio=0
    avg_nb_non_zero=0
    if expected>0:
        avg_nb_non_zero=sum(nb_non_zero)/len(nb_non_zero)
        ratio=expected/tagged*100
    elif tagged==0:
        ratio='NaN'
        avg_nb_non_zero='NaN'
    dataset_results.append([dataset_name,population,tagged,expected,others,avg_nb_non_zero])
#    display(HTML(
#'<table><tr>{}</tr></table>'.format(
#    '</tr><tr>'.join(
#        '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in matched)
#)))
    dataset_number+=1

display(HTML(
'<hr/><h1>SUMMARY</h1><br/><table><tr>{}</tr></table>'.format(
    '</tr><tr>'.join(
        '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in dataset_results)
)))

2016-05-26.tab 280 280
2016-05-18.tab 199 199
2016-05-28.tab 2 2
2016-05-29.tab 4 4
2016-06-07.tab 14 14
2016-06-06.tab 74 74
2016-05-11.tab 155 155
2016-05-21.tab 2 2
2016-06-01.tab 219 219
2016-05-13.tab 63 63
2016-05-25.tab 185 185
2016-05-27.tab 134 134
2016-05-20.tab 99 99
2016-05-22.tab 1 1
2016-05-10.tab 112 112
2016-06-04.tab 17 17
2016-05-24.tab 5 5
2016-05-16.tab 5 5
2016-05-12.tab 71 71
2016-05-30.tab 292 292
2016-05-23.tab 58 58
2016-05-31.tab 187 187
2016-06-02.tab 331 331
2016-06-05.tab 18 18
2016-06-03.tab 164 164
2016-05-17.tab 48 48
2016-05-19.tab 87 87


logfile,events,attack_events,detected,others_detections,nb_dim_err
2016-05-26.tab,4177,613,0,11,0
2016-05-18.tab,25553,0,0,1,NaN
2016-05-28.tab,2924,2920,0,0,0
2016-05-29.tab,2918,2909,0,0,0
2016-06-07.tab,2224,0,0,0,NaN
2016-06-06.tab,27006,53,0,0,0
2016-05-11.tab,557,0,0,0,NaN
2016-05-21.tab,3,0,0,0,NaN
2016-06-01.tab,83599,4,0,9,0
2016-05-13.tab,157,0,0,1,NaN
